In [7]:
import torch
import torch.nn as nn

In [134]:
class Mogrifier_LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, future_step, batch_size, layer_size = 2):
        super(Mogrifier_LSTM, self).__init__()

        self.input_size = input_size
        self.hidden_size = hidden_size
        self.future_step = future_step
        self.batch_size = batch_size

        self.mogrifier_lstm_layer1 = MogrifierLSTMCell(input_size, hidden_size, layer_size)
        self.mogrifier_lstm_layer2 = MogrifierLSTMCell(hidden_size, hidden_size, layer_size)

        # Backward LSTM layer
        self.mogrifier_lstm_layer1_backward = MogrifierLSTMCell(input_size, hidden_size, layer_size)
        self.mogrifier_lstm_layer2_backward = MogrifierLSTMCell(hidden_size, hidden_size, layer_size)

        self.linear1 = nn.Linear(hidden_size, hidden_size)
        self.linear = nn.Linear(hidden_size * lag_size, future_step * input_size)
        self.relu = nn.ReLU()
        self.flatten = nn.Flatten()
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):

        h1, c1 = [torch.zeros(self.batch_size, self.hidden_size).to(device), torch.zeros(self.batch_size, self.hidden_size).to(device)]
        h2, c2 = [torch.zeros(self.batch_size, self.hidden_size).to(device), torch.zeros(self.batch_size, self.hidden_size).to(device)]
        
        h1_backward, c1_backward = torch.zeros(self.batch_size, self.hidden_size).to(device), torch.zeros(self.batch_size, self.hidden_size).to(device)
        h2_backward, c2_backward = torch.zeros(self.batch_size, self.hidden_size).to(device), torch.zeros(self.batch_size, self.hidden_size).to(device)

        hidden_states_forward = []
        hidden_states_backward = []

        outputs = []
        for i in range(lag_size):
            tempx = x[:, i]
            h1,c1 = self.mogrifier_lstm_layer1(tempx, (h1, c1))     
            #h2,c2 = self.mogrifier_lstm_layer2(h1, (h2, c2))
            #h1 = self.dropout(h1)
            #out = self.linear1(h1)
            hidden_states_forward.append(h1.unsqueeze(1))
            #outputs.append(out.unsqueeze(1)) 

        # for i in reversed(range(x.size(1))):
        #     tempx = x[:, i]
        #     h1_backward, c1_backward = self.mogrifier_lstm_layer1_backward(tempx, (h1_backward, c1_backward))     
        #     #h2_backward, c2_backward = self.mogrifier_lstm_layer2_backward(h1_backward, (h2_backward, c2_backward))
        #     #out = self.linear1(h1_backward)
        #     hidden_states_backward.append(h2.unsqueeze(1))
        hidden_states_forward = torch.cat(hidden_states_forward, dim=1)
        # hidden_states_backward = torch.cat(hidden_states_backward, dim=1)

        # Concatenate forward and backward hidden states
        # hidden_states = torch.cat((hidden_states_forward, hidden_states_backward), dim=-1)
        #hidden_states = torch.flatten(hidden_states[:, -1, :])
        outputs = self.linear(hidden_states_forward.view(self.batch_size, -1))  # Only take the last time step output
        #outputs = self.linear(hidden_states_forward[:, -1, :])  # Only take the last time step output
        outputs = outputs.view(self.batch_size, self.future_step, -1)
        # hidden_states = torch.cat(hidden_states, dim = 1)  
        # outputs = torch.cat(outputs, dim = 1)  
        # outputs = outputs[:, -1, :]     
        
        return self.relu(outputs)

In [135]:
class MogrifierLSTMCell(nn.Module):

    def __init__(self, input_size, hidden_size, mogrify_steps):
        super(MogrifierLSTMCell, self).__init__()
        self.mogrify_steps = mogrify_steps
        self.lstm = nn.LSTMCell(input_size, hidden_size)
        self.mogrifier_list = nn.ModuleList([nn.Linear(hidden_size, input_size)])
        for i in range(1, mogrify_steps):
            if i % 2 == 0:
                self.mogrifier_list.extend([nn.Linear(hidden_size, input_size)])
            else:
                self.mogrifier_list.extend([nn.Linear(input_size, hidden_size)])
   
    def mogrify(self, x, h):
        for i in range(self.mogrify_steps):
            if (i+1) % 2 == 0: 
                h = (2*torch.sigmoid(self.mogrifier_list[i](x))) * h
            else:
                x = (2*torch.sigmoid(self.mogrifier_list[i](h))) * x
        return x, h

    def forward(self, x, states):
        ht, ct = states
        x, ht = self.mogrify(x, ht)
        ht, ct = self.lstm(x, (ht, ct))
        return ht, ct